# M2 Rigid Body Position Small range
 
This Jupyter notebook performs the Rigid Body Position (RBP) test of the M2 in the context of the M2 minimum functionality check.
Each M2 RBP Degree of Freedom (DoF) is actuated individually with RBP command of small amplitude and the evolution of the measured forces is observed. A conditional loop ensures that the maximum forces and related errors remain below the fault threshold.

## Import Modules

In [ ]:
import asyncio
import yaml
import numpy as np
import pandas as pd
import sys
import select
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK
from lsst.ts.ofc.utils import get_config_dir as get_config_dir_ofc
from lsst.ts.aos.utils import DiagnosticsM2, EfdName
import time

## Prepare the M2 CSC and put to Enabled state

In [ ]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")

In [ ]:
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

# SYSTEM STATUS LEGEND    
1. DISABLED = 1
2. ENABLED = 2
3. FAULT = 3
4. OFFLINE = 4
5. STANDBY = 5

Standby  -->  Disable  -->  Enabled

await m2.cmd_start.set_start(timeout=30)
await m2.cmd_enable.set_start(timeout=200)

To Stop, do this:    Enabled  -->  Disable  -->  Standby

await m2.cmd_disable.set_start(timeout=30)
await m2.cmd_standby.set_start(timeout=30) 

In [ ]:
# get status
state = m2.evt_summaryState.get()
print(f'{time.strftime("%Y-%m-%dT%H:%M:%S",time.gmtime())} System Status: {state.summaryState}')

In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=30)

## LUT FILES
1. Configurable_File_Description_20180831T091922_M2_optical.csv
2. Configurable_File_Description_20180831T092326_M2_handling.csv
3. Configurable_File_Description_20180831T092423_surrogate_optical.csv
4. Configurable_File_Description_20180831T092556_surrogate_handling.csv

In [ ]:
#Setting the right LUT ***ONLY IN DISABLE STATE ***
ccfile = m2.evt_config.get().get_vars()['configuration']

if 'surrogate_optical' not in ccfile:
    config_files = m2.evt_configurationFiles.get().get_vars()['files'].split(',')
    ncfile = [el for el in config_files if 'surrogate_optical' in el][0]
    await m2.cmd_setConfigurationFile.set_start(file=ncfile)

print(f'{time.strftime("%Y-%m-%dT%H:%M:%S",time.gmtime())} Current config file: {m2.evt_config.get().get_vars()["configuration"]}')

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=450)

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=460)

## Command RBP displacement

In [ ]:
async def move_m2_rbp(axis: str):

    start = await asyncio.to_thread(input, 'Start')
    match start:
        case 'y':
            pass
        case _:
            return
            
    position = await asyncio.to_thread(input, 'Position')
    if len(position) == 0:
        return
    position = float(position)    

    # ABSOLUTE position displacement
    match axis:
        case 'x': 
            await m2.cmd_positionMirror.set_start(x=position)
        case 'y': 
            await m2.cmd_positionMirror.set_start(y=position)
        case 'z': 
            await m2.cmd_positionMirror.set_start(z=position)
        case _: 
            raise ValueError
        
   
    #awaiting until the position is stable
    interval = 5.0
    threshold = 2.0
    mirror_is_moving = True
    while mirror_is_moving:
       
        dof_meas = list()
        ti = time.perf_counter()
        while(time.perf_counter() - ti < interval):
            match axis:
                case 'x':
                    dof_meas.append(m2.tel_position.get().x)
                case 'y':
                    dof_meas.append(m2.tel_position.get().y)
                case 'z':
                    dof_meas.append(m2.tel_position.get().z)
                case _:
                    raise ValueError
               
        if abs(max(dof_meas)-min(dof_meas)) < threshold:
            mirror_is_moving = False
        await asyncio.sleep(5.0)


    # force threshold checks
    tangent_force_error = m2.tel_forceErrorTangent.get().force
    measured_tangent = m2.tel_tangentForce.get().measured
    measured_axial = m2.tel_axialForce.get().measured

    
    tangent_force_error = np.abs(np.array(tangent_force_error)).max()
    measured_tangent = np.abs(np.array(measured_tangent)).max()
    measured_axial = np.abs(np.array(measured_axial)).max()

    sum_force = abs(m2.tel_forceErrorTangent.get().sum)
    weight_force = abs(m2.tel_forceErrorTangent.get().weight)

    # if the threshold is exceeded, then break
    if (tangent_force_error > 950 or measured_tangent > 4870 or
    sum_force > 950 or weight_force > 1900 or measured_axial > 420):
        print('Some limit reached:')
        print(f'{time.strftime("%Y-%m-%dT%H:%M:%S",time.gmtime())}\nFmeasured: {measured_tangent} \nError_tangent: {tangent_force_error} \nSum: {sum_force} \nWeight: {weight_force} \nMeasured_axial: {measured_axial}')
        return
        
    print(f'{time.strftime("%Y-%m-%dT%H:%M:%S",time.gmtime())} \nFmeasured: {measured_tangent} \nError_tangent: {tangent_force_error} \nSum: {sum_force} \nWeight: {weight_force} \nMeasured_axial: {measured_axial}')
    print('*** MOVEMENT DONE ***')
    await move_m2_rbp(axis)

In [ ]:
#Tiemout coroutine
async def timeout(timeout: float):
    await asyncio.sleep(timeout)
    return

In [ ]:
async def main(axis):
    t = 60.0

    task1 = asyncio.create_task(timeout(t))
    task2 = asyncio.create_task(move_m2_rbp(axis))

    done, pending = await asyncio.wait([task1, task2], return_when=asyncio.FIRST_COMPLETED)

In [ ]:
# X-DOF

axis='x'

await m2.cmd_positionMirror.set_start() #Restoring Zero position
await main(axis)

In [ ]:
# Y-DOF

axis='y'

await m2.cmd_positionMirror.set_start() #Restoring Zero position
await main(axis)

In [ ]:
# Z-DOF

axis='z'

await m2.cmd_positionMirror.set_start() #Restoring Zero position
await main(axis)